<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#라이브러리-로드" data-toc-modified-id="라이브러리-로드-1">라이브러리 로드</a></span></li><li><span><a href="#데이터-준비" data-toc-modified-id="데이터-준비-2">데이터 준비</a></span><ul class="toc-item"><li><span><a href="#데이터셋-저장" data-toc-modified-id="데이터셋-저장-2.1">데이터셋 저장</a></span></li><li><span><a href="#폴더-생성-및-압축-해제" data-toc-modified-id="폴더-생성-및-압축-해제-2.2">폴더 생성 및 압축 해제</a></span></li><li><span><a href="#이미지-resize-함수-정의" data-toc-modified-id="이미지-resize-함수-정의-2.3">이미지 resize 함수 정의</a></span></li><li><span><a href="#Train-dataset-리사이즈" data-toc-modified-id="Train-dataset-리사이즈-2.4">Train dataset 리사이즈</a></span></li><li><span><a href="#Test-dataset-리사이즈" data-toc-modified-id="Test-dataset-리사이즈-2.5">Test dataset 리사이즈</a></span></li><li><span><a href="#tf.dataset-만들기" data-toc-modified-id="tf.dataset-만들기-2.6">tf.dataset 만들기</a></span></li></ul></li><li><span><a href="#모델-정의" data-toc-modified-id="모델-정의-3">모델 정의</a></span><ul class="toc-item"><li><span><a href="#모델-훈련" data-toc-modified-id="모델-훈련-3.1">모델 훈련</a></span></li><li><span><a href="#모델-평가-및-결론" data-toc-modified-id="모델-평가-및-결론-3.2">모델 평가 및 결론</a></span></li></ul></li><li><span><a href="#모델-개선" data-toc-modified-id="모델-개선-4">모델 개선</a></span><ul class="toc-item"><li><span><a href="#새로운-데이터셋-생성" data-toc-modified-id="새로운-데이터셋-생성-4.1">새로운 데이터셋 생성</a></span></li><li><span><a href="#transparent-and-resize" data-toc-modified-id="transparent-and-resize-4.2">transparent and resize</a></span><ul class="toc-item"><li><span><a href="#이미지-투명하게-만들기" data-toc-modified-id="이미지-투명하게-만들기-4.2.1">이미지 투명하게 만들기</a></span></li><li><span><a href="#배경-이미지-png로-저장하기" data-toc-modified-id="배경-이미지-png로-저장하기-4.2.2">배경 이미지 png로 저장하기</a></span></li><li><span><a href="#이미지-경로-저장" data-toc-modified-id="이미지-경로-저장-4.2.3">이미지 경로 저장</a></span></li><li><span><a href="#resize,-rotate,-flip-함수-작성" data-toc-modified-id="resize,-rotate,-flip-함수-작성-4.2.4">resize, rotate, flip 함수 작성</a></span></li></ul></li><li><span><a href="#tf.dataset-만들기" data-toc-modified-id="tf.dataset-만들기-4.3">tf.dataset 만들기</a></span><ul class="toc-item"><li><span><a href="#만들어진-데이터셋-예시" data-toc-modified-id="만들어진-데이터셋-예시-4.3.1">만들어진 데이터셋 예시</a></span></li><li><span><a href="#Train,-Valid-set-나누기" data-toc-modified-id="Train,-Valid-set-나누기-4.3.2">Train, Valid set 나누기</a></span></li></ul></li><li><span><a href="#모델-정의" data-toc-modified-id="모델-정의-4.4">모델 정의</a></span><ul class="toc-item"><li><span><a href="#Resnet101-Pretrained-모델-사용" data-toc-modified-id="Resnet101-Pretrained-모델-사용-4.4.1">Resnet101 Pretrained 모델 사용</a></span></li></ul></li><li><span><a href="#결론" data-toc-modified-id="결론-4.5">결론</a></span></li></ul></li></ul></div>

# 라이브러리 로드

In [ ]:
import glob
import os
from pathlib import Path

import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import Image as display

AUTOTUNE = tf.data.AUTOTUNE
cwd = Path(os.getcwd())

In [ ]:
classes = ['scissor', 'rock', 'paper']

# 데이터 준비

https://teachablemachine.withgoogle.com/
  
위 사이트에 들어가서 데이터를 준비하면 됩니다.  
예시 사진은 아래와 같습니다.

In [ ]:
display("src/01/1.png", width="500", height="300")

## 데이터셋 저장
`data/01` 경로 안에 paper.zip, scissor.zip, rock.zip 파일을 준비합니다.

In [ ]:
display("src/01/2.png", width="300", height="100")

## 폴더 생성 및 압축 해제

In [ ]:
!mkdir data/01/paper
!mkdir data/01/scissor
!mkdir data/01/rock

!unzip data/01/paper.zip -d data/01/paper
!unzip data/01/scissor.zip -d data/01/scissor
!unzip data/01/rock.zip -d data/01/rock

## 이미지 resize 함수 정의

In [ ]:
def resize_images(img_path, save_path, target_size=(28,28)):
    if not os.path.isdir(save_path):
        os.makedirs(save_path)
    
    images= list(img_path.glob("*.jpg"))
    
    print(len(images), " images to be resized.")

    for img in images:
        filename = os.path.basename(img)
        old_img=Image.open(img)
        new_img=old_img.resize(target_size, Image.ANTIALIAS)
        new_img.save(save_path / filename, "JPEG")

    print(len(images), " images resized.")   


## Train dataset 리사이즈

In [ ]:
for class_ in classes:
    origin_img_path = cwd / 'data/01' / class_
    save_path = cwd / 'data/01/resize' / class_
    resize_images(origin_img_path, save_path)
    print(class_, '작업완료', '\n')

## Test dataset 리사이즈

In [ ]:
for class_ in classes:
    cwd = Path(os.getcwd())
    origin_img_path = cwd / 'data/01/test' / class_
    save_path = cwd / 'data/01/resize/test' / class_
    resize_images(origin_img_path, save_path)
    print(class_, '작업완료', '\n')

## tf.dataset 만들기

In [ ]:
def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    one_hot = parts[-2] == classes
    return tf.argmax(one_hot)

def decode_img(img):
    img = tf.io.decode_jpeg(img, channels=3)
    return img

def process_path(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label


In [ ]:
paths = []
for class_ in classes:
    data_path = cwd / 'data/01/resize' / class_
    paths += list(data_path.glob("*.jpg"))
paths = [str(_) for _ in paths]
list_ds = tf.data.Dataset.list_files(paths, shuffle=False)
train_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE).shuffle(300).batch(32)

In [ ]:
test_paths = []
for class_ in classes:
    data_path = cwd / 'data/01/resize/test' / class_
    test_paths += list(data_path.glob("*.jpg"))
test_paths = [str(_) for _ in test_paths]

test_ds = tf.data.Dataset.list_files(test_paths, shuffle=False)
test_ds = test_ds.map(process_path, num_parallel_calls=AUTOTUNE).batch(32)

# 모델 정의

In [ ]:
model=keras.models.Sequential()
model.add(keras.layers.Conv2D(8, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(16, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(3, activation='softmax'))

model.summary()

In [ ]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

## 모델 훈련

In [ ]:
model.fit(train_ds, epochs=10)

In [ ]:
model.evaluate(test_ds)

## 모델 평가 및 결론

결과
- train set 정확도는 99.67% 를 달성했지만, test set 정확도는 16.67%에 불과하다.

이유
- Overfitting이 발생했다.
- 데이터가 충분하지 않다.
- 각 데이터가 모두 비슷하다.

=> 새로운 손 데이터 셋을 찾아보자.  
=> 그리고 각 이미지 샘플의 배경을 다르게 해보자.

# 모델 개선

<img src="https://i.ibb.co/vLqmP8z/4.png" width="500"/>

- **기본 아이디어**  
1. 데이터  
위 사진과 같이, 손 그림의 흰 영역을 투명으로 처리한 후, 배경이미지 위에 올려 다양한 배경 위에 손이 존재하도록 한다.  
 이 때 손은 회전과 크기조절을 랜덤하게 적용한다.  
 배경 또한 300장의 이미지를 랜덤하게 적용한다.

  
2. 모델
pretrained model을 사용하자

## 새로운 데이터셋 생성

아래 사이트에서 가위바위보 손모양 데이터셋 다운로드 받기  
https://laurencemoroney.com/datasets.html#rock-paper-scissors-dataset  
**압축 파일을 풀고, scissors 폴더 이름을 scissor 로 바꿔주세요**

<img src="https://i.ibb.co/z4S9KSS/2021-09-28-01-55-14.png" width="400"/>

배경으로 쓸 사진 다운로드 받기  
https://storage.googleapis.com/openimages/web/download.html

<img src="https://storage.googleapis.com/openimages/web/images/fiftyone.png" width="500"/>

**data/01 폴더 안에 hand 폴더를 만들고, 위에서 받은 손모양 이미지들을 준비해주세요.  
data/01 폴더 안에 background 폴더를 만들고, 위에서 받은 배경 이미지들을 준비해주세요.**

In [ ]:
display("src/01/7.png", width="200", height="150")

## transparent and resize

### 이미지 투명하게 만들기

In [ ]:
from PIL import Image

def make_transparent(img_path, save_path):
    if not os.path.isdir(save_path):
        os.makedirs(save_path)
    
    images= list(img_path.glob("*.png"))
    for img in images:
        filename = os.path.basename(img)
        img = Image.open(img)
        img = img.convert("RGBA")
        datas = img.getdata()
        newData = []
        for item in datas:
            if 225 <= item[0] <= 255 and 225 <= item[1] <= 255 and 225 <= item[2] <= 255:
                newData.append((255, 255, 255, 0))
            else:
                newData.append(item)
        img.putdata(newData)
        img.save(save_path / filename, "PNG")

In [ ]:
for class_ in classes:
    img_path = cwd / 'data/01/hand' / class_
    save_path = cwd / 'data/01/resize/hand' / class_
    make_transparent(img_path, save_path)

### 배경 이미지 png로 저장하기

In [ ]:
def save_png(img_path, save_path, target_size=(224, 224)):
    if not os.path.isdir(save_path):
        os.makedirs(save_path)
    
    images= list(img_path.glob("*.jpg"))
    
    print(len(images), " images to be resized.")

    for img in images:
        filename = os.path.basename(img) + '.png'
        img = Image.open(img)
        width, height = img.size
        img = img.resize(target_size)
        img = img.convert("RGBA")
        img.save(save_path / filename, "PNG")

    print(len(images), " images resized.")   


In [ ]:
img_path = cwd / 'data/01/background'
save_path = cwd / 'data/01/resize/background'
save_png(img_path, save_path)

### 이미지 경로 저장

In [ ]:
background_paths = cwd / 'data/01/resize/background'
background_paths =  list(background_paths.glob("*.png"))
background_paths = [str(_) for _ in background_paths]

In [ ]:
hand_paths = []
for class_ in classes:
    data_path = cwd / 'data/01/resize/hand' / class_
    hand_paths += list(data_path.glob("*.png"))
hand_paths = [str(_) for _ in hand_paths]

### resize, rotate, flip 함수 작성

In [ ]:
def resize(img, prob, ratio):
    w, h = img.size
    if np.random.rand() < prob:
        ratio = max(ratio, np.random.rand())
        w = int(w*ratio)
        h = int(h*ratio)
        img = img.resize((w, h))
    return img


def rotate(img, prob):
    if np.random.rand() < prob:
        theta = int(np.random.rand() * 90)
        img = img.rotate(angle=theta)
    return img


def flip(img, prob):
    if np.random.rand() < prob:
        if np.random.rand() < 0.5:
            img = np.flip(img, 0)
        else:
            img = np.flip(img, 1)
    return img


def image_merge(back, front):
    parts = tf.strings.split(front, os.path.sep)
    one_hot = parts[-2] == classes
    label = tf.argmax(one_hot)
    label = tf.cast(label, tf.int8)
    
    back = back.numpy()
    front = front.numpy()
    background = Image.open(back)
    foreground = Image.open(front)
    
    foreground = resize(foreground, 0.5, 0.5)
    foreground = rotate(foreground, 0.5)
    
    
    background.paste(foreground, (0, 0), foreground)
    background = background.convert("RGB")
    background = np.array(background)
    background = flip(background, 0.5)
    
    
    return background, label


## tf.dataset 만들기

In [ ]:
train_ds_hand = tf.data.Dataset.list_files(hand_paths).shuffle(300)
train_ds_background = tf.data.Dataset.list_files(background_paths).shuffle(300)
train_ds = tf.data.Dataset.zip((train_ds_background, train_ds_hand))

In [ ]:
def tf_img_merge(back, front):
    image, label = tf.py_function(image_merge, [back, front], [tf.int16, tf.int8])
    image.set_shape((224,224,3))
    label.set_shape(())
    return image, label

In [ ]:
img_merge_ds = train_ds.map(tf_img_merge)
img_merge_ds = img_merge_ds.batch(32)

In [ ]:
# for a in img_merge_ds.take(1):
#     plt.imshow(a[0][0].numpy())
#     plt.show()

### 만들어진 데이터셋 예시

<img src="https://i.ibb.co/ZSYPS7y/6.png" width="500"/>

### Train, Valid set 나누기

In [ ]:
def is_test(x, y):
    return x % 4 == 0

def is_train(x, y):
    return not is_test(x, y)

recover = lambda x,y: y

test_dataset = img_merge_ds.enumerate().filter(is_test).map(recover)
train_dataset = img_merge_ds.enumerate().filter(is_train).map(recover)

## 모델 정의

### Resnet101 Pretrained 모델 사용

In [ ]:
base_model = keras.applications.ResNet101(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

In [ ]:
# freeze 한 후, 모델 훈련

base_model.trainable = False

In [ ]:
inputs = keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(3, activation='softmax')(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_dataset, epochs=10, validation_data=test_dataset)

In [ ]:
# unfreeze 한 후, 모델 훈련

base_model.trainable = True

model.compile(optimizer=keras.optimizers.Adam(1e-5),  # Very low learning rate
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_dataset, epochs=10, validation_data=test_dataset)

In [ ]:
test_paths = []
for class_ in classes:
    data_path = cwd / 'data/01/test' / class_
    test_paths += list(data_path.glob("*.jpg"))
test_paths = [str(_) for _ in test_paths]

test_ds = tf.data.Dataset.list_files(test_paths, shuffle=False)
test_ds = test_ds.map(process_path).batch(12)

In [ ]:
model.evaluate(test_ds)

In [ ]:
pred = model.predict(test_ds)
np.argmax(pred, axis = -1)

In [ ]:
y = [j for i, j in test_ds]
y

## 결론
정확도 16.67%에서 83.33% 로 상승 

아래 사진을 주먹으로 인식하는 에러 존재

In [ ]:
display("src/01/8.png", width="400", height="200")